In [37]:
from google.cloud import bigquery
from BigQueryHelper import BigQueryHelper as bqh
import os
import pandas as pd
import json
import numpy as np

In [38]:
pathToRoot = '../../../'
client = bigquery.Client.from_service_account_json(os.path.join(pathToRoot, 'credentials/rescribe-nlp-creds.txt'))
datasetLength = 10000

In [39]:
data = bqh(active_project="bigquery-public-data",
            dataset_name="stackoverflow", 
            client=client)

data.list_tables()

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [40]:
#Ok, so what we want are the query titles, the query descriptions, the code from the top answer, and the tags associated with it
#Note: individual tables in this dataset blow past 30 gigs

QUERY = f"""
SELECT 
    id, title, body, accepted_answer_id, tags
FROM 
    bigquery-public-data.stackoverflow.posts_questions
WHERE accepted_answer_id IS NOT NULL AND tags IS NOT NULL
LIMIT {datasetLength}

"""
questions = data.query_to_pandas(QUERY)

In [41]:
print(questions.dtypes)
print(f"Number of rows: {len(questions)}")
questions.sample(5)

id                     int64
title                 object
body                  object
accepted_answer_id     int64
tags                  object
dtype: object
Number of rows: 10000


,id,title,body,accepted_answer_id,tags
6107,61952927,How PigStorage is used in Hadoop and Why?,<p>I got a quite confused as why we need anoth...,61964910,hadoop|apache-pig
9942,61990435,How we can Iterate within a particular row wit...,<p>I have a data frame named df_cp which has t...,61990641,python|pandas
9944,61996477,"Pandas apply to create multiple columns, using...",<p>I am trying to use a function to create mul...,61996488,python|pandas
9203,62073725,Repeatedly compare same portion of dataset to ...,<p>I have a dataframe that looks like the foll...,62073752,r|loops|dplyr|apply|tidyverse
8574,61792745,How to build new object using keys from array ...,<p>I have this array of keys:</p>\n\n<p><code>...,61793248,javascript|arrays|ecmascript-6|javascript-objects


In [42]:
with open(os.path.join(pathToRoot,"datasets/post-questions.json"), 'w') as outfile: 
    json.dump(questions.to_json(), outfile)

In [43]:
#alright, time for the answers
QUERY = f"""
SELECT 
    id, body, parent_id
FROM 
    bigquery-public-data.stackoverflow.posts_answers
WHERE id IN {tuple(questions.accepted_answer_id.values)} AND parent_id IS NOT NULL
LIMIT {datasetLength}
"""

answers = data.query_to_pandas(QUERY)

In [44]:
print(answers.dtypes)
print(f"Number of rows: {len(answers)}")
answers.sample(5)

id            int64
body         object
parent_id     int64
dtype: object
Number of rows: 10000


,id,body,parent_id
9807,61801746,<p>This:</p>\n\n<pre><code>df.loc[df['GDP'].id...,61801654
4878,62065862,<p>Neither Spring Data R2DBC nor R2DBC Postgre...,62042888
650,61981121,<p>Never got why the above XML gave me all 0's...,61830044
1346,61844655,<p>Your code has some seriously flawed logic. ...,61824735
6636,61866442,<p>What you are looking for is the <code>flex-...,61860652


In [45]:
with open(os.path.join(pathToRoot,"datasets/post-answers.json"), 'w') as outfile: 
    json.dump(answers.to_json(), outfile)

In [46]:
#and now time for the tags
QUERY = f"""
SELECT
    id, tag_name, count, excerpt_post_id, wiki_post_id
FROM 
     bigquery-public-data.stackoverflow.tags
WHERE excerpt_post_id IS NOT NULL AND wiki_post_id IS NOT NULL
ORDER BY 
    count DESC
LIMIT {datasetLength}
"""

tags = data.query_to_pandas(QUERY)

In [47]:
# tags = tags.astype({'excerpt_post_id': np.int64, 'wiki_post_id': np.int64}, errors='ignore')
print(tags.dtypes)
print(f"Number of rows: {len(tags)}")
tags.head(5)

id                  int64
tag_name           object
count               int64
excerpt_post_id     int64
wiki_post_id        int64
dtype: object
Number of rows: 10000


,id,tag_name,count,excerpt_post_id,wiki_post_id
0,3,javascript,2018353,3624960,3607052
1,17,java,1678103,3624966,3607018
2,16,python,1444269,3624965,3607014
3,9,c#,1409903,3624962,3607007
4,5,php,1353178,3624936,3607050


In [48]:
with open(os.path.join(pathToRoot, "datasets/tags.json"), 'w') as outfile:
    json.dump(tags.to_json(), outfile)

In [49]:
#and now for the tag exerpt, this is a shortened version of the wiki. 
QUERY = f"""
SELECT
  id, body
FROM 
  bigquery-public-data.stackoverflow.posts_tag_wiki_excerpt
WHERE id IN {tuple(tags.excerpt_post_id.values)}
LIMIT {datasetLength}

"""

tag_excerpts = data.query_to_pandas(QUERY)

In [50]:
print(tag_excerpts.dtypes)
print(f"Number of rows: {len(tag_excerpts)}")
tag_excerpts.sample(5)

id       int64
body    object
dtype: object
Number of rows: 10000


,id,body
524,47895109,Font Awesome 5. This tag is for questions rela...
5038,14332231,The Kendo Grid is a Kendo widget that displays...
8235,6053980,A weak reference is a reference that does not ...
798,18325721,Intrinsics functions are used in compiled lang...
9970,46791950,None


In [51]:
with open(os.path.join(pathToRoot, "datasets/tag-excerpts.json"), 'w') as outfile:
    json.dump(tag_excerpts.to_json(), outfile)

In [52]:
#and now lastly for the tag wiki
QUERY = f"""
SELECT
  id, body
FROM 
  bigquery-public-data.stackoverflow.posts_tag_wiki
WHERE id IN {tuple(tags.wiki_post_id.values)}
LIMIT {datasetLength}

"""

tag_wikis = data.query_to_pandas(QUERY)

In [53]:
print(tag_wikis.dtypes)
print(f"Number of rows: {len(tag_wiki)}")
tag_wikis.sample(5)

id       int64
body    object
dtype: object


NameError: name 'tag_wiki' is not defined

In [54]:
with open(os.path.join(pathToRoot, "datasets/tag-wikis.json"), 'w') as outfile:
    json.dump(tag_wikis.to_json(), outfile)